# Beautiful soup


In [22]:
from bs4 import BeautifulSoup
import requests
import re

def get_pages_barri(soup):
    pags=soup.find_all('div',class_="pagination")
    extension=list(set(re.findall(r'pagina-\d*.htm',str(pags))))
    return extension

def get_info_container(soup):
    container=soup.find_all('div', class_='item-info-container')
    prices=[]
    sizes=[]
    Ids=[]

    for item in container:

        item=str(item)
        price=re.findall(r'<span class="item-price">\d+.?\d+.?\d+',item)[0][25:]
        price=str.replace(price,'.','')
        size=re.findall(r'<span class="item-detail">\d+.?\d+ <small>m²',item)[0][26:-10]
        size=str.replace(size,'.','')
        Id=re.findall(r'href="/inmueble/\d*',item)[0][16:]
        Ids.append(Id)
        sizes.append(size)
        prices.append(price)
    return Ids,prices,sizes

def get_next_page(soup):
    pags=soup.find_all(class_="icon-arrow-right-after")
    if len(pags) != 0:
        return re.findall(r'pagina-\d*.htm',str(pags))[0]
    else:
        return 'stop'

In [27]:
page = requests.get("https://www.idealista.com/alquiler-viviendas/barcelona/sant-andreu/baro-de-viver/")
print ('good' if page.status_code==200 else 'bad') #200 is good
soup = BeautifulSoup(page.content, 'html.parser')
soup

good


<html lang="es" xml:lang="es" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>idealista.com : pisos madrid, pisos barcelona, pisos valencia. inmobiliarias venta, alquiler y obra nueva</title>
<meta content="text/html; charset=utf-8" http-equiv="content-Type"/>
<meta content="0" http-equiv="Expires"/>
<meta content="no-cache" name="Pragma"/>
<meta content="no-cache" http-equiv="pragma"/>
<meta content="no-cache, mustrevalidate" http-equiv="cache-control">
<meta content="NOODP" name="ROBOTS"/>
<style type="text/css">
        html,body,div,span,h3,p,a,img,strong,ol,ul,li{border:0 none;font-family:inherit;font-size:100.01%;
        font-style:normal;font-weight:inherit;margin:0;outline-color:invert;
        outline-style:none;outline-width:0;padding:0;vertical-align:baseline;}
        ol,ul{list-style-image:none;list-style-position:outside;list-style-type:none;}
        body {position:relative;background:#fff;width:960px;color:#333;font:12px Verdana,Geneva,Arial,Helvetica,sans-serif;te

In [36]:
pags=soup.find_all(class_="icon-arrow-right-after")
len(pags)
re.findall(r'pagina-\d*.htm',str(pags))[0]


'pagina-2.htm'

In [31]:
import pandas as pd
import numpy as np
file=pd.read_csv('url-neighborhood.csv')
# tipus='venta'
tipus='alquiler'
file_means=open('./output-'+tipus+'/means.dat','w')
file_means.write('district,neightborhood,mean-price(€),mean-size(m^2),mean-price-size(€/m^2)\n')
for distric,nei,path in zip(file['District'],file['Neighborhood'],file['Idealista-path']):

    print(1)    
    page = requests.get("https://www.idealista.com/"+tipus+"-viviendas/barcelona/"+str(path))
    soup = BeautifulSoup(page.content, 'html.parser')

    next_pages=get_pages_barri(soup)
    ID,Price,Size=get_info_container(soup)
    ID=np.array(ID,dtype=int)
    Price=np.array(Price,dtype=int)
    Size=np.array(Size,dtype=int)
    next_page=get_next_page(soup)
    while next_page !='stop':
#     for next_page in next_pages:
        print(next_page)
        page = requests.get("https://www.idealista.com/"+tipus+"-viviendas/barcelona/"+str(path)+str(next_page))
        soup = BeautifulSoup(page.content, 'html.parser')
        next_pages=get_pages_barri(soup)
        ID_,Price_,Size_=get_info_container(soup)

        ID=np.r_[ID,np.array(ID_,dtype=int)]
        Price=np.r_[Price,np.array(Price_,dtype=int)]
        Size=np.r_[Size,np.array(Size_,dtype=int)]
        next_page=get_next_page(soup)
        
    data=np.c_[ID.T,Price.T,Size.T]
    data=pd.DataFrame(data,columns=['ID','PRICE(€)','SIZE(m^2)'])
    pd.DataFrame.to_csv(data,'./output-'+tipus+'/zone'+str(distric)+'-'+str(nei)+'.csv',index=False)
    file_means.write('{},{},{},{},{}\n'.format(distric,nei,np.mean(Price),np.mean(Size),np.mean(Price/Size)))
file_means.close()   

1
pagina-2.htm
pagina-3.htm
pagina-4.htm
pagina-5.htm
pagina-6.htm
pagina-7.htm
pagina-8.htm
pagina-9.htm
pagina-10.htm
pagina-11.htm
pagina-12.htm
pagina-13.htm
pagina-14.htm
pagina-15.htm
pagina-16.htm
pagina-17.htm
pagina-18.htm
pagina-19.htm
pagina-20.htm
pagina-21.htm
pagina-22.htm
pagina-23.htm
pagina-24.htm
pagina-25.htm
pagina-26.htm
pagina-27.htm
pagina-28.htm
1
pagina-2.htm
pagina-3.htm
pagina-4.htm
pagina-5.htm
pagina-6.htm
pagina-7.htm
pagina-8.htm
pagina-9.htm
pagina-10.htm
pagina-11.htm
pagina-12.htm
pagina-13.htm
pagina-14.htm
pagina-15.htm
pagina-16.htm
1
pagina-2.htm
pagina-3.htm
pagina-4.htm
pagina-5.htm
1
pagina-2.htm
pagina-3.htm
pagina-4.htm
pagina-5.htm
1
pagina-2.htm
pagina-3.htm
pagina-4.htm
pagina-5.htm
pagina-6.htm
1
pagina-2.htm
pagina-3.htm
pagina-4.htm
pagina-5.htm
pagina-6.htm
pagina-7.htm
pagina-8.htm
pagina-9.htm
pagina-10.htm
pagina-11.htm
pagina-12.htm
pagina-13.htm
pagina-14.htm
pagina-15.htm
pagina-16.htm


In [28]:
file_means.close()

In [2]:
url_alquiler='https://www.idealista.com/alquiler-viviendas/barcelona/sants-montjuic/sants-badal/'
page=requests.get(url_alquiler)

In [50]:
print ('good' if page.status_code==200 else 'bad') #200 is good
soup = BeautifulSoup(page.content, 'html.parser')
print(get_pages_barri(soup))
print(get_info_container(soup))

good
['pagina-3.htm', 'pagina-2.htm']
(['38778457', '38476642', '38777759', '38559458', '38776418', '38775095', '38773320', '38771885', '38469533', '38712990', '32897413', '37931423', '38153797', '33085324', '38108980', '38744112', '38534537', '38617675', '38556456', '38151087', '38153449', '38677705', '38676813', '25483123', '38171320', '35283802', '38664723', '38267822', '38151897', '35283855'], ['1100', '1600', '765', '900', '1250', '1300', '1000', '1000', '875', '1000', '1375', '900', '1100', '825', '1300', '1100', '900', '790', '875', '1100', '1100', '1200', '1800', '1100', '1000', '1200', '1350', '1440', '1000', '1200'], ['66', '97', '45', '45', '71', '75', '70', '70', '85', '67', '100', '65', '65', '72', '50', '82', '70', '68', '70', '90', '65', '120', '98', '80', '90', '60', '119', '95', '95', '50'])


In [56]:
ide,preu,tamany=get_info_container(soup)

In [72]:
import numpy as np
import pandas as pd
ide=np.array(ide,dtype=int)
preu=np.array(preu,dtype=int)
tamany=np.array(tamany,dtype=int)
data=np.c_[ide.T,preu.T,tamany.T]
data=pd.DataFrame(data,columns=['ID','PRICE(€)','SIZE(m^2)'])
pd.DataFrame.to_csv(data,'prova.csv',index=False)